<a href="https://colab.research.google.com/github/negovanovic-e/automatska-kategorizacija-proizvoda/blob/main/04_skripte_i_produkcija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Direktorij za skripte
import os
if not os.path.exists('scripts'):
    os.makedirs('scripts')

# SKRIPTA 1: train_model.py
train_script = '''
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import joblib
import os

def preprocess_text(text):
    """Funkcija za preprocesiranje teksta naziva proizvoda"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\\w\\s]', ' ', text)
    text = re.sub(r'\\d+', '', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

def train_model():
    """Glavna funkcija za treniranje modela"""
    print(" POČINJEM TRENIRANJE MODELA...")

    # Učitavanje podataka
    df = pd.read_csv('products.csv')
    print(f"Učitano {len(df)} proizvoda")

    # Čišćenje naziva stupaca
    df.columns = df.columns.str.strip()

    # Uklanjanje redova s nedostajućim podacima
    df = df.dropna(subset=['Product Title', 'Category Label'])
    print(f" Nakon čišćenja: {len(df)} proizvoda")

    # Preprocesiranje teksta
    df['cleaned_title'] = df['Product Title'].apply(preprocess_text)

    # Inženjering značajki
    df['title_length'] = df['cleaned_title'].apply(len)
    df['word_count'] = df['cleaned_title'].apply(lambda x: len(x.split()))

    # TF-IDF vektorizacija
    tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
    X_tfidf = tfidf.fit_transform(df['cleaned_title'])

    # Kombinacija značajki
    X_combined = hstack([X_tfidf, df[['title_length', 'word_count']].values])
    y = df['Category Label']

    # Podjela podataka
    X_train, X_test, y_train, y_test = train_test_split(
        X_combined, y, test_size=0.2, random_state=42, stratify=y
    )

    # Treniranje modela
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluacija
    accuracy = model.score(X_test, y_test)
    print(f"TOČNOST MODELA: {accuracy:.2%}")

    # Kreiranje direktorija za modele
    if not os.path.exists('models'):
        os.makedirs('models')

    # Spremanje modela
    joblib.dump(model, 'models/product_classifier.pkl')
    joblib.dump(tfidf, 'models/tfidf_vectorizer.pkl')

    print(" MODEL JE USPIJEŠNO SPREMLJEN!")

if __name__ == "__main__":
    train_model()
'''

# Spremi skriptu
with open('scripts/train_model.py', 'w') as f:
    f.write(train_script)

print("✅ train_model.py je kreiran u folderu 'scripts'")

✅ train_model.py je kreiran u folderu 'scripts'


In [2]:
# SKRIPTA 2: predict_category.py
predict_script = '''
import joblib
import re
import pandas as pd
from scipy.sparse import hstack

def preprocess_text(text):
    """Funkcija za preprocesiranje teksta"""
    text = str(text).lower()
    text = re.sub(r'[^\\w\\s]', ' ', text)
    text = re.sub(r'\\d+', '', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    return text

def predict_category(product_title):
    """Funkcija za predikciju kategorije proizvoda"""
    try:
        # Učitavanje modela i vektorizera
        model = joblib.load('models/product_classifier.pkl')
        tfidf = joblib.load('models/tfidf_vectorizer.pkl')

        # Preprocesiranje unesenog teksta
        cleaned_title = preprocess_text(product_title)

        # Kreiranje značajki
        title_length = len(cleaned_title)
        word_count = len(cleaned_title.split())

        # TF-IDF transformacija
        X_tfidf = tfidf.transform([cleaned_title])

        # Kombinacija značajki
        X_combined = hstack([X_tfidf, [[title_length, word_count]]])

        # Predikcija
        prediction = model.predict(X_combined)
        probability = model.predict_proba(X_combined).max()

        return prediction[0], probability
    except FileNotFoundError:
        return "Model nije pronađen. Prvo pokrenite train_model.py", 0.0
    except Exception as e:
        return f"Greška: {e}", 0.0

def main():
    """Glavna funkcija za interaktivno testiranje"""
    print("=== SUSTAV ZA AUTOMATSKU KLASIFIKACIJU PROIZVODA ===")
    print("Unesite naziv proizvoda za klasifikaciju (ili 'quit' za izlaz):")

    while True:
        user_input = input("\\n Naziv proizvoda: ")

        if user_input.lower() == 'quit':
            print(" Hvala na korištenju!")
            break

        if user_input.strip() == "":
            print(" Molimo unesite naziv proizvoda.")
            continue

        category, confidence = predict_category(user_input)
        print(f"Predložena kategorija: {category}")
        print(f"Vjerojatnost: {confidence:.2%}")

if __name__ == "__main__":
    main()
'''

with open('scripts/predict_category.py', 'w') as f:
    f.write(predict_script)

print("✅ predict_category.py je kreiran u folderu 'scripts'")

✅ predict_category.py je kreiran u folderu 'scripts'
